# Abfragen und Visualsierungen 

>In diesem Notebook werden mit verschiedenen Abfragen und Visualisierungen unterschiedliche Information aus der Datenbank entnommen. Die Abfragen der Datenbank erfolgen über pymongo und werden teilweise mit pandas weiterverarbeitet. Die Visualisierungen werden sowohl mit plotly express, wie auch folium umgesetzt. 

Folgende Fragestellungen werden im folgenden beantwortet: 
* Query 1: Wie hat sich die Summe der verbrannten Flächen weltweit entwickelt? 
* Query 2: Welche Länder sind besonders stark betroffen? 
* Query 3: Wie verteilt sich die verbrannte Fläche weltweit auf die verschiedenen Vegetationsklassen? 
* Query 4: In welchen Gegenden von Deutschland ist Fläche größer als ein Quadradkilometer verbrannt?

In [155]:
from pymongo import MongoClient
import pandas as pd
import plotly.express as px
import folium
import pandas as pd
import pycountry
import plotly.io as pio

pio.templates.default = "plotly_white"


In [138]:
#!"C:\Program Files\MongoDB\Server\6.0\bin\mongod.exe" --dbpath "C:\mongodb_data"

In [139]:
client = MongoClient()
client = MongoClient('mongodb://localhost:27017/')
mydb = client["esa_fire"]
mycol = mydb["esa_fire_2001"]
print(mydb.name)

esa_fire


## Query 1: Wie hat sich die Summe der verbrannten Flächen weltweit entwickelt? 

In [143]:
list_date = list ()
agg_result= mycol.aggregate(
    [{
    "$group" : 
        {"_id" : "$date", 
         "burned_area" : {"$sum" : 1}
         }
    }])
for i in agg_result:
    list_date.append(i)

In [144]:
df_date = pd.DataFrame(list_date)
df_date = df_date.sort_values(by='_id')

In [176]:
fig = px.bar(df_date, x="_id", y="burned_area", title='Life expectancy in Canada', color_discrete_sequence=px.colors.qualitative.Prism)
fig.show()

Auf den ersten Blick auf die Grafik fallen die fehlenden Daten in 2019 auf. Die Ursache dafür liegt in der Datenquelle. Aufgrund eines Fehler sind die Datena uf der Website von Oktober, November und Dezember 2019 in einem seperaten Ordner. 

Bei genauerer Bedrachtung der Grafik fehlt auf, dass jedes Jahr eine ähnliche Waldbrandentwicklung auf weist. Die meiste verbrannte Fläche findet sich in den Sommermonaten und am niedrigsten sind die Werte im Winter. Ebenfalls lässt sich die Entwicklung der Summe der verbrannte Fläche auf Monatsebene schwierig beurteilen, deswegen ist ein weitere Aggregation auf Jahr sinnvoll. 

In [177]:
df_date['year'] = df_date['_id'].str.slice(start=0, stop=4)
df_groupdate = df_date.groupby(by='year').agg(month=('_id', 'first'), sum=('burned_area', 'sum'))
df_groupdate.reset_index(inplace= True)
df_groupdate

,year,month,sum
0,2001,2001-01-01,137834
1,2002,2002-01-01,167281
2,2003,2003-01-01,175374
3,2004,2004-01-01,174907
4,2005,2005-01-01,178833
5,2006,2006-01-01,172256
6,2007,2007-01-01,177836
7,2008,2008-01-01,174292
8,2009,2009-01-01,162955
9,2010,2010-01-01,163529


In [178]:
fig = px.line(df_groupdate, x='month', y='sum',  title='Life expectancy in Canada', color_discrete_sequence=px.colors.qualitative.Prism)
fig.show()

TODO Urasache erklärenBrandrodung?

## Query 2: Welche Länder sind besonders stark betroffen? 

In [148]:
list_date = list ()
agg_result= mycol.aggregate(
    [{
    "$group" : 
        {"_id" : "$country", 
         "burned_area" : {"$sum" : 1}
         }
    }])
for i in agg_result:
    list_date.append(i)

In [149]:
df_countries = pd.DataFrame(list_date).sort_values(by=['burned_area'], ascending= False)
df_countries = df_countries.head(20)

In [150]:
df_countries['name'] = df_countries['_id'].apply(lambda x: pycountry.countries.get(alpha_2=x).name)

In [179]:
fig = px.bar(df_countries, x="name", y="burned_area",  title='Life expectancy in Canada', color_discrete_sequence=px.colors.qualitative.Prism)
fig.show()

## Query 3: Wie verteilt sich die verbrannte Fläche weltweit auf die verschiedenen Vegetationsklassen? 

In [159]:

list_veg = list ()
agg_result= mycol.aggregate([{ "$group":
  { "_id" : None,  
    "burned_area"  : { "$sum": "$burned_area" }, 
    "no_data_burned_area"  : { "$sum":"$no_data_burned_area" },
    "tree_cover_broadleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_broadleaved_deciduous_burned_area" },
    "tree_cover_broadleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_broadleaved_deciduous_burned_area" },
    "tree_cover_needleleaved_evergreen_burned_area"  : { "$sum":"$tree_cover_needleleaved_evergreen_burned_area" },
    "tree_cover_needleleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_needleleaved_deciduous_burned_area" },
    "tree_cover_mixed_leave_burned_area"  : { "$sum":"$tree_cover_mixed_leave_burned_area"},
    "cropland_burned_area"  : { "$sum":"$cropland_burned_area" },
    "mosaic_tree_grass_burned_area"  : { "$sum":"$mosaic_tree_grass_burned_area" },
    "flooded_area_burned_area"  : { "$sum":"$flooded_area_burned_area" },
    "other_vegetation_burned_area"  : { "$sum":"$other_vegetation_burned_area" },}
}])
for i in agg_result:
    list_veg.append(i)
list_veg

[{'_id': None,
  'burned_area': 89940704892432.0,
  'no_data_burned_area': 7004942864842.0,
  'tree_cover_broadleaved_deciduous_burned_area': 35183926581228.0,
  'tree_cover_needleleaved_evergreen_burned_area': 807798707721.0,
  'tree_cover_needleleaved_deciduous_burned_area': 988045537799.0,
  'tree_cover_mixed_leave_burned_area': 151480670524.0,
  'cropland_burned_area': 5687008018231.0,
  'mosaic_tree_grass_burned_area': 4188674581006.0,
  'flooded_area_burned_area': 2544568553482.0,
  'other_vegetation_burned_area': 31979288438507.0}]

In [160]:
df_veg = pd.DataFrame(list_veg)

In [161]:
df_veg= df_veg.drop(columns='_id')

In [181]:
import plotly.graph_objects as go

fig1  = go.Figure()
hrz = ["burned_area", "no_data_burned_area","tree_cover_broadleaved_deciduous_burned_area",
       "tree_cover_needleleaved_evergreen_burned_area","tree_cover_needleleaved_deciduous_burned_area", "cropland_burned_area", "mosaic_tree_grass_burned_area", "flooded_area_burned_area", "other_vegetation_burned_area"
       "Final"]
vrt = [ df_veg["burned_area"][0], 
       -df_veg["no_data_burned_area"][0],
       -df_veg["tree_cover_broadleaved_deciduous_burned_area"][0],
       -df_veg["tree_cover_needleleaved_evergreen_burned_area"][0],
       -df_veg["tree_cover_needleleaved_deciduous_burned_area"][0],
       -df_veg["cropland_burned_area"][0],
       -df_veg["mosaic_tree_grass_burned_area"][0],
       -df_veg["flooded_area_burned_area"][0],
       -df_veg["other_vegetation_burned_area"][0],
       None]
fig1.add_trace(go.Waterfall(                 
                  x = hrz, y = vrt,
                  base = 0,
                  measure = [ "absolute","relative",    
                              "relative","relative",
                              "relative","relative" ], 
              decreasing = {"marker":{"color":'rgb(95,70,144)'}},
              totals = {"marker":{"color":'rgb(56,165,166)'}},                    
                ),)                                 
fig1.show()



## Query 4: In welchen Gegenden von Deutschland ist Fläche größer als ein Quadradkilometer verbrannt?

In [140]:
# Querys Mongo DB
query2={
     '$and': [
            { 'country' : 'DE' },
            { 'burned_area'    : { '$gt': 1000000 }}
          ]
}
points =mycol.find(query2)
print("Anzahl der Pixel in Deutschland mit einer verbrannten Fläche größer als 1km2: ",mycol.count_documents(query2))

Anzahl der Pixel in Deutschland mit einer verbrannten Fläche größer als 1km2:  43


In [141]:
m = folium.Map(location=[51.02700,10.27486],zoom_start=7)
for citie in points:
    folium.Marker( location=[ citie['lat'], citie['lon']], radius=8,tooltip=citie['date'], icon = folium.Icon(color="red", icon='fire')).add_to(m)

Die Punkte mit einer verbrannten Fläche größer als 1km2 werden auf der Karte mit Markern mit "Feuersymbolen" dargestellt. Mit einem Mouse-Over über die Punkte werden die Daten der Brände angezeigt. 

In [142]:
#m